# Analytics Layer (Rearc Quest) — BLS × DataUSA Population

This notebook implements the **analytics requirements** using the raw datasets landed by the ingestion steps:

- **BLS raw file (Part 1):** `/Volumes/rearc_quest/lakehouse/raw_bls/pr.data.0.Current`
- **Population raw JSON (Part 2):** `/Volumes/rearc_quest/lakehouse/raw_datausa/population.json`

All computations are performed with **Spark (PySpark)** and results are materialized as **Delta tables** in:
`rearc_quest.lakehouse`

---

### Requirement A — Population summary (2013–2018)
Compute **mean** and **standard deviation** of population for years **2013–2018**.

Output Delta table:
- `rearc_quest.lakehouse.population_stats_2013_2018`

---

### Requirement B — Best year per BLS series
For each `series_id`, compute the **best year** defined as the year with the **maximum yearly sum** of values.

Output Delta table:
- `rearc_quest.lakehouse.bls_best_year_by_series`

---

### Requirement C — Joined report (BLS × Population)
Create a report for:
- `series_id = PRS30006032`
- `period = Q01`

Joined with **population by year**.

Output Delta table:
- `rearc_quest.lakehouse.report_prs30006032_q01`

---

## Design 

- **Raw → Analytics**: This notebook treats ingestion outputs as immutable raw sources and produces curated analytics tables downstream.
- **Idempotent outputs**: All tables are written in `overwrite` mode for deterministic re-runs.
- **Data quality checks**:
  - Filters blank lines from BLS raw text
  - Validates population JSON has a non-empty `data` array
- **Extensibility**: The same patterns (parse → normalize → aggregate → publish) can be extended to additional BLS series, other measures, or year ranges.

---

## How to run
1. Ensure Part 1 and Part 2 ingestion notebooks have successfully populated the raw paths.
2. Run cells top to bottom.
3. Validate outputs via:
   - `SHOW TABLES IN rearc_quest.lakehouse;`
   - `SELECT * FROM rearc_quest.lakehouse.report_prs30006032_q01 ORDER BY year;`


## Setup + paths + output tables

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import json

# ------------------------------------------------------------------------------
# Setup: set catalog/schema, define (catalog/schema) 
# ------------------------------------------------------------------------------
CATALOG = "rearc_quest"
SCHEMA  = "lakehouse"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

# ------------------------------------------------------------------------------
# Inputs from ingestion steps
# - BLS file is a fixed "latest snapshot" landing from Part 1
# - population.json is a raw snapshot landing from Part 2
# ------------------------------------------------------------------------------
BLS_FILE = f"/Volumes/{CATALOG}/{SCHEMA}/raw_bls/pr.data.0.Current"
POP_JSON = f"/Volumes/{CATALOG}/{SCHEMA}/raw_datausa/population.json"

# ------------------------------------------------------------------------------
# Analytics outputs are captured into Delta tables
# ------------------------------------------------------------------------------
OUT_STATS   = f"{CATALOG}.{SCHEMA}.population_stats_2013_2018"
OUT_BESTYR  = f"{CATALOG}.{SCHEMA}.bls_best_year_by_series"
OUT_REPORT  = f"{CATALOG}.{SCHEMA}.report_prs30006032_q01"

print("Using BLS file:", BLS_FILE)
print("Using population file:", POP_JSON)


Using BLS file: /Volumes/rearc_quest/lakehouse/raw_bls/pr.data.0.Current
Using population file: /Volumes/rearc_quest/lakehouse/raw_datausa/population.json


## Parse BLS raw file into a structured DataFrame

In [0]:
# ------------------------------------------------------------------------------
# Read BLS fixed-width/space-delimited text
# Format (simplified): series_id year period value
# We treat the raw file as append-only snapshot content and normalize into columns
# ------------------------------------------------------------------------------
bls_raw = spark.read.text(BLS_FILE)

# Split on whitespace; trim first to avoid empty tokens
parts = F.split(F.trim(F.col("value")), r"\s+")

bls = (
    bls_raw
    # Remove blank / whitespace-only lines
    .where(F.length(F.trim("value")) > 0)
    .select(
        parts.getItem(0).alias("series_id"),
        parts.getItem(1).try_cast("int").alias("year"),
        parts.getItem(2).alias("period"),
        parts.getItem(3).cast("double").alias("value")
    )
    # Filters: ensure essential fields exist
    .where(
        F.col("series_id").isNotNull() &
        F.col("year").isNotNull() &
        F.col("period").isNotNull() &
        F.col("value").isNotNull()
    )
)

display(bls.limit(10))


series_id,year,period,value
PRS30006011,1995,Q01,2.6
PRS30006011,1995,Q02,2.1
PRS30006011,1995,Q03,0.9
PRS30006011,1995,Q04,0.1
PRS30006011,1995,Q05,1.4
PRS30006011,1996,Q01,-0.2
PRS30006011,1996,Q02,-0.3
PRS30006011,1996,Q03,-0.1
PRS30006011,1996,Q04,0.2
PRS30006011,1996,Q05,-0.1


## Read population JSON (DataUSA) and normalize

In [0]:
# ------------------------------------------------------------------------------
# population.json structure from DataUSA:
# { annotations: {...}, columns: [...], data: [ {Year:..., Population:...}, ... ] }
# We only extract Year + Population for analytics joins.
# ------------------------------------------------------------------------------
raw = dbutils.fs.head(POP_JSON, 20_000_000)
payload = json.loads(raw)

rows = payload.get("data", [])
if not rows:
    raise RuntimeError("population.json has no 'data' records")

population = (
    spark.createDataFrame(rows)
    .withColumn("Year", F.col("Year").cast("int"))
    .withColumn("Population", F.col("Population").cast("double"))
    .select("Year", "Population")
    .where(F.col("Year").isNotNull() & F.col("Population").isNotNull())
)

display(population.orderBy("Year"))


Year,Population
2013,3.16128839E8
2014,3.18857056E8
2015,3.21418821E8
2016,3.23127515E8
2017,3.25719178E8
2018,3.27167439E8
2019,3.28239523E8
2021,3.31893745E8
2022,3.33287562E8
2023,3.34914896E8


## Requirement A — Population mean + stddev (2013–2018)

In [0]:
# ------------------------------------------------------------------------------
# Requirement A:
# Mean + standard deviation of population for years 2013–2018 (inclusive)
# ------------------------------------------------------------------------------
pop_stats = (
    population
    .where((F.col("Year") >= 2013) & (F.col("Year") <= 2018))
    .agg(
        F.mean("Population").alias("mean_population"),
        F.stddev("Population").alias("stddev_population")
    )
)

display(pop_stats)

# Publish as a  Delta table (deterministic overwrite)
pop_stats.write.mode("overwrite").format("delta").saveAsTable(OUT_STATS)


mean_population,stddev_population
3.22069808E8,4158441.040908095


## Requirement B — Best year per series_id (max yearly sum)

In [0]:
# ------------------------------------------------------------------------------
# Requirement B:
# For each series_id, aggregate values by year and select the year with max sum.
# Tie-breaker: if same summed_value, pick earliest year (stable/deterministic).
# ------------------------------------------------------------------------------
yearly_sum = (
    bls
    .groupBy("series_id", "year")
    .agg(F.sum("value").alias("summed_value"))
)

w = Window.partitionBy("series_id").orderBy(
    F.col("summed_value").desc(),
    F.col("year").asc()
)

best_year = (
    yearly_sum
    .withColumn("rn", F.row_number().over(w))
    .where(F.col("rn") == 1)
    .select("series_id", F.col("year").alias("best_year"), "summed_value")
)

display(best_year)

best_year.write.mode("overwrite").format("delta").saveAsTable(OUT_BESTYR)


series_id,best_year,summed_value
PRS30006011,2022,20.5
PRS30006012,2022,17.1
PRS30006013,1998,705.895
PRS30006021,2010,17.7
PRS30006022,2010,12.399999999999999
PRS30006023,2014,503.21600000000007
PRS30006031,2022,20.5
PRS30006032,2021,17.1
PRS30006033,1998,702.672
PRS30006061,2022,34.5


## Requirement C — Join PRS30006032 Q01 with population by year

In [0]:
# ------------------------------------------------------------------------------
# Requirement C:
# Filter the BLS dataset to series PRS30006032 and period Q01,
# then join population by year for a combined report.
# ------------------------------------------------------------------------------
report = (
    bls
    .where((F.col("series_id") == "PRS30006032") & (F.col("period") == "Q01"))
    .select("series_id", "year", "period", "value")
    .join(
        population.select(
            F.col("Year").alias("year"),
            F.col("Population").alias("population")
        ),
        on="year",
        how="left"     # left join keeps BLS rows even if population year missing
    )
    .orderBy("year")
)

display(report)

report.write.mode("overwrite").format("delta").saveAsTable(OUT_REPORT)


year,series_id,period,value,population
1995,PRS30006032,Q01,0.0,null
1996,PRS30006032,Q01,-4.2,null
1997,PRS30006032,Q01,2.8,null
1998,PRS30006032,Q01,0.9,null
1999,PRS30006032,Q01,-4.1,null
2000,PRS30006032,Q01,0.5,null
2001,PRS30006032,Q01,-6.3,null
2002,PRS30006032,Q01,-6.6,null
2003,PRS30006032,Q01,-5.7,null
2004,PRS30006032,Q01,2.0,null


## Validation cell

In [0]:
from pyspark.sql import functions as F
import json

def assert_path_exists(p: str):
    try:
        dbutils.fs.ls(p)
        print(f"Found: {p}")
    except Exception as e:
        raise RuntimeError(f" Missing required path: {p}\n{e}")

def assert_table_exists(full_name: str):
    try:
        spark.table(full_name)
        print(f" Table exists: {full_name}")
    except Exception as e:
        raise RuntimeError(f" Missing required table: {full_name}\n{e}")

# ---- 1) Validate inputs ----
print("\n========== VALIDATION: INPUTS ==========")
assert_path_exists(BLS_FILE)
assert_path_exists(POP_JSON)

# Quick sanity: population JSON has data array
pop_head = dbutils.fs.head(POP_JSON, 5_000_000)
payload = json.loads(pop_head)
if not payload.get("data"):
    raise RuntimeError(" population.json is present but has empty 'data' array")
print(" population.json contains records:", len(payload["data"]))

# Quick sanity: BLS parsed row counts
print("\n========== VALIDATION: PARSED DATAFRAMES ==========")
print("BLS rows:", bls.count())
print("Population rows:", population.count())

# Basic checks
bad_bls = bls.where(
    F.col("series_id").isNull() | F.col("year").isNull() | F.col("period").isNull() | F.col("value").isNull()
).count()
if bad_bls > 0:
    raise RuntimeError(f" BLS parse produced {bad_bls} rows with null critical fields")
print("BLS parse critical fields OK")

# ---- 2) Validate required outputs ----
print("\n========== VALIDATION: OUTPUT TABLES ==========")
assert_table_exists(OUT_STATS)
assert_table_exists(OUT_BESTYR)
assert_table_exists(OUT_REPORT)

# ---- 3) Show concise sanity samples ----
print("\n========== OUTPUT SAMPLES ==========")
print("Population stats:")
display(spark.table(OUT_STATS))

print("Best year by series (sample 10):")
display(spark.table(OUT_BESTYR).orderBy(F.col("summed_value").desc()).limit(10))

print("Report PRS30006032 Q01:")
display(spark.table(OUT_REPORT).orderBy("year"))



========== VALIDATION: INPUTS ==========
Found: /Volumes/rearc_quest/lakehouse/raw_bls/pr.data.0.Current
Found: /Volumes/rearc_quest/lakehouse/raw_datausa/population.json
 population.json contains records: 10

========== VALIDATION: PARSED DATAFRAMES ==========
BLS rows: 37521
Population rows: 10
BLS parse critical fields OK

========== VALIDATION: OUTPUT TABLES ==========
 Table exists: rearc_quest.lakehouse.population_stats_2013_2018
 Table exists: rearc_quest.lakehouse.bls_best_year_by_series
 Table exists: rearc_quest.lakehouse.report_prs30006032_q01

========== OUTPUT SAMPLES ==========
Population stats:


mean_population,stddev_population
3.22069808E8,4158441.040908092


Best year by series (sample 10):


series_id,best_year,summed_value
PRS88003183,2024,1047.3410000000001
PRS88003193,2024,862.564
PRS88003083,2024,791.0899999999999
PRS84006083,2024,786.344
PRS85006083,2024,785.831
PRS88003053,2024,759.825
PRS84006053,2024,754.179
PRS85006053,2024,753.9569999999999
PRS88003063,2024,737.752
PRS85006063,2024,729.423


Report PRS30006032 Q01:


year,series_id,period,value,population
1995,PRS30006032,Q01,0.0,null
1996,PRS30006032,Q01,-4.2,null
1997,PRS30006032,Q01,2.8,null
1998,PRS30006032,Q01,0.9,null
1999,PRS30006032,Q01,-4.1,null
2000,PRS30006032,Q01,0.5,null
2001,PRS30006032,Q01,-6.3,null
2002,PRS30006032,Q01,-6.6,null
2003,PRS30006032,Q01,-5.7,null
2004,PRS30006032,Q01,2.0,null
